<a href="https://colab.research.google.com/github/keringKirwa/4.1_Machine_Learning/blob/main/Final_Year_DistilBERT_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the datasets library
!pip install datasets

# Install spaCy version 3.2.2
!pip install spacy==3.2.2

# Install the spaCy model
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0.tar.gz

!pip install pdfplumber

! pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl

!pip install spacy==3.2.2

!pip install spacy-transformers

!pip install seqeval

!pip install nest_asyncio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
  Attempting uninstall: pydantic
    Found e

In [ ]:
import spacy
print(spacy.__version__)

3.2.2


In [ ]:
import gdown
import zipfile
import os

In [ ]:
file_id = '1ifrNKggpHB5jSSBKHqSCMy3jOJ1WD1mC'

# Defining the directory paths
base_dir = '/content/drive/MyDrive/ColabNotebooks'
zip_file_path = os.path.join(base_dir, 'case_files.zip')
unzip_dir = os.path.join(base_dir, 'case_files')

# Create base directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [ ]:
# Download the file
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, zip_file_path, quiet=False)

# Create the directory to unzip the files
if not os.path.exists(unzip_dir):
    os.makedirs(unzip_dir)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)


Downloading...
From (original): https://drive.google.com/uc?id=1ifrNKggpHB5jSSBKHqSCMy3jOJ1WD1mC
From (redirected): https://drive.google.com/uc?id=1ifrNKggpHB5jSSBKHqSCMy3jOJ1WD1mC&confirm=t&uuid=a418234f-c264-47d2-8825-575611067c2c
To: /content/drive/MyDrive/ColabNotebooks/case_files.zip
100%|██████████| 237M/237M [00:01<00:00, 146MB/s]


In [ ]:
land_disputes_dir = os.path.join(unzip_dir, 'land disputes')


file_names = os.listdir(land_disputes_dir)



num_files = len(file_names)

print(f'Number of files in "land disputes": {num_files}')


Number of files in "land disputes": 3743


In [ ]:
import spacy
import spacy_transformers
import torch
import datasets
import numpy as np
from datasets import Dataset, ClassLabel, Features, Sequence,Value
from transformers import AutoTokenizer, AutoModelForTokenClassification

torch.set_default_dtype(torch.float32)

In [ ]:
sbd_model = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [ ]:
def detect_and_disambiguate_sentences(text):
    text = text.replace("\n", " ")
    doc = sbd_model(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

In [ ]:
print(file_names[0])

Environment & Land Case 95 of 2007.pdf


In [ ]:
import os
import pdfplumber

def pdf_to_text(file_path):
    """
    Convert PDF file to text.

    Args:
    - file_path (str): Path to the PDF file.

    Returns:
    - str: Text content of the PDF file.
    """
    try:
        with pdfplumber.open(file_path) as pdf:
            if pdf:
                # Extract text from each page
                text = ""
                for page in pdf.pages:
                    text += page.extract_text()
                return text
            else:
                print(f"Error: Could not open {file_path}. It may not be a valid PDF.")
                return None
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None

def generator(relative_folder_path = land_disputes_dir):
    for file_name in os.listdir(relative_folder_path):

        if file_name.endswith(".pdf"):  # Checking if the file is a .pdf file
            relative_file_path = os.path.join(relative_folder_path, file_name)

            file_contents = pdf_to_text(relative_file_path)

            if file_contents:
                sentences = detect_and_disambiguate_sentences(file_contents)

                for s in sentences:
                    yield s.strip()



In [ ]:
out_sample= pdf_to_text(os.path.join(land_disputes_dir, file_names[1]))
print(len(detect_and_disambiguate_sentences(out_sample)))


80


In [ ]:
nlp = spacy.load("en_legal_ner_trf")
text = "Section 319 Cr.P.C. contemplates a situation where the evidence adduced by the prosecution for Respondent No.3-G. Sambiah on 20th June 1984"
doc = nlp(text)

# Print indentified entites
for ent in doc.ents:
     print(ent,ent.label_)



Section 319 PROVISION
Cr.P.C. STATUTE
G. Sambiah RESPONDENT
20th June 1984 DATE


In [ ]:
CSI_NER_Base_Model = spacy.load("en_legal_ner_trf")

In [ ]:
def convert_to_dictionary(data):
    tokens = [item[0] for item in data]
    ner_tags = [item[1] for item in data]
    return {'tokens': tokens, 'ner_tags': ner_tags}


In [ ]:
def create_example(sbd_sentence):
    doc = CSI_NER_Base_Model(sbd_sentence)
    IOB_tuples = []

    # Iterate through spaCy span entities and create CoNLL-formatted annotations
    for span in doc.ents:
        index_of_the_first_token = span.start
        index_of_the_last_token = span.end
        label_ = span.label_  # PERSON , ORG, LOC ,O,STATUTE

        for i in range(index_of_the_first_token, index_of_the_last_token):
            word = doc[i].text
            if i == index_of_the_first_token:
                label = "B-" + label_
            else:
                label = "I-" + label_
            IOB_tuples.append((word, label))

    adjusted_IOB_tuples = []
    iob_labels = [tup[1] for tup in IOB_tuples]

    pointer_index = 0
    for token in doc:
        word = token.text
        if pointer_index < len(iob_labels) and word.lower() == IOB_tuples[pointer_index][0].lower():
            label = iob_labels[pointer_index]
            pointer_index += 1
        else:
            label = "OUT"
        adjusted_IOB_tuples.append((word, label))

    result = convert_to_dictionary(adjusted_IOB_tuples)
    return result


In [ ]:
sample_example = "Section 319 Cr.P.C. contemplates a situation where the evidence adduced by the prosecution for Respondent No.3-G. Sambiah on 20th June 1984 on 12/05/2020"

iob_annotations = create_example(sample_example)
print(len(iob_annotations['tokens']), len(iob_annotations['ner_tags']))

27 27


In [ ]:
from itertools import islice
from tqdm import tqdm

examples = []

In [ ]:
import asyncio
import concurrent.futures
from itertools import islice
from tqdm.notebook import tqdm
import nest_asyncio

nest_asyncio.apply()

sentence_generator = generator(land_disputes_dir)
examples = []  # Initialize examples as a global variable

async def create_example_parallel(sentence):
    loop = asyncio.get_event_loop()
    result = await loop.run_in_executor(None, lambda: create_example(sentence))
    return result

async def main():
    global examples  # Use the global examples variable

    num_files = len(file_names)
    print(f"The number of files: {num_files}")

    # Using ThreadPoolExecutor to parallelize the execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        tasks = [create_example_parallel(sentence) for sentence in islice(sentence_generator,2000)]
        completed_tasks = []

        # Wait for all tasks to complete
        results = await asyncio.gather(*tasks)

        for result in tqdm(results, total=len(results)):
            try:
                examples.append(result)
                completed_tasks.append(result)
            except Exception as e:
                print(f"An error occurred: {e}")

    num_examples = len(examples)
    print(f"Total Examples generated: {num_examples}")

    # Check the number of completed tasks
    print(f"Number of completed tasks: {len(completed_tasks)}")

# Run the main coroutine
await main()


The number of files: 3743
An error occurred while processing /content/drive/MyDrive/ColabNotebooks/case_files/land disputes/Environment & Land Case 95 of 2007.pdf: No /Root object! - Is this really a PDF?
An error occurred while processing /content/drive/MyDrive/ColabNotebooks/case_files/land disputes/Environment & Land 244 of 2014.pdf: No /Root object! - Is this really a PDF?
An error occurred while processing /content/drive/MyDrive/ColabNotebooks/case_files/land disputes/Environment and Land Case 643 of 2014.pdf: No /Root object! - Is this really a PDF?
An error occurred while processing /content/drive/MyDrive/ColabNotebooks/case_files/land disputes/Environment & Land Civil Case 175 of 2011.pdf: No /Root object! - Is this really a PDF?
An error occurred while processing /content/drive/MyDrive/ColabNotebooks/case_files/land disputes/Environment and Land Petition 3 of 2019.pdf: No /Root object! - Is this really a PDF?
An error occurred while processing /content/drive/MyDrive/ColabNoteb

  0%|          | 0/2000 [00:00<?, ?it/s]

Total Examples generated: 2000
Number of completed tasks: 2000


In [ ]:
examples[0]

{'tokens': ['In',
  're',
  'Estate',
  'of',
  'Iddi',
  'Makokha',
  'Mwima',
  'alias',
  'Makokha',
  'Mwima',
  '(',
  'Deceased',
  ')',
  '[',
  '2021',
  ']',
  'eKLR',
  'REPUBLIC',
  'OF',
  'KENYA',
  'IN',
  'THE',
  'HIGH',
  'COURT',
  'OF',
  'KENYA',
  'AT',
  'KAKAMEGA',
  'SUCCESSION',
  'CAUSE',
  'NO',
  '.',
  '661',
  'OF',
  '2013',
  'IN',
  'THE',
  'MATTER',
  'OF',
  'THE',
  'ESTATE',
  'OF',
  'IDDI',
  'MAKOKHA',
  'MWIMA',
  'alias',
  'MAKOKHA',
  'MWIMA',
  '(',
  'DECEASED',
  ')',
  'RULING',
  '1',
  '.'],
 'ner_tags': ['OUT',
  'OUT',
  'OUT',
  'OUT',
  'B-OTHER_PERSON',
  'I-OTHER_PERSON',
  'I-OTHER_PERSON',
  'I-OTHER_PERSON',
  'I-OTHER_PERSON',
  'I-OTHER_PERSON',
  'OUT',
  'OUT',
  'OUT',
  'OUT',
  'OUT',
  'OUT',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'OUT',
  'OUT',
  'B-COURT',
  'I-COURT',
  'I-COURT',
  'I-COURT',
  'I-COURT',
  'I-COURT',
  'OUT',
  'B-CASE_NUMBER',
  'I-CASE_NUMBER',
  'I-CASE_NUMBER',
  'I-CASE_NUMBER',
  'I-

In [ ]:
import json

classes = {
    "OUT": 0,
    "B-RESPONDENT": 11,
    "I-RESPONDENT": 12,
    "B-COURT": 21,
    "I-COURT": 22,
    "B-PETITIONER": 31,
    "I-PETITIONER": 32,
    "B-PROVISION": 41,
    "I-PROVISION": 42,
    "B-LAWYER": 51,
    "I-LAWYER": 52,
    "B-STATUTE": 61,
    "I-STATUTE": 62,
    "B-CASE_NUMBER": 71,
    "I-CASE_NUMBER": 72,
    "B-DATE": 81,
    "I-DATE": 82,
    "B-OTHER_PERSON": 91,
    "I-OTHER_PERSON": 92,
    "B-JUDGE": 103,
    "I-JUDGE": 104,
    "B-ORG": 113,
    "I-ORG": 114,
    "B-GPE": 123,
    "I-GPE": 124,
    "B-WITNESS": 133,
    "I-WITNESS": 134,
    "B-PRECEDENT": 143,
    "I-PRECEDENT": 144
}



json_file_path = json_file_path = os.path.join(base_dir, 'entity_classes.json')
with open(json_file_path, "w") as json_file:
    json.dump(classes, json_file)


In [ ]:
with open(json_file_path, "r") as json_file:
    classes = json.load(json_file)

NUM_CLASSES = len(classes.keys())
print(f"Number of classes: {NUM_CLASSES}")


Number of classes: 29


# New Section

In [ ]:
features_dict = {
    "tokens":  Sequence(
        feature= Value(dtype='string', id=None),
        length=-1,
        id=None
    ),
    "ner_tags":Sequence(
        feature= ClassLabel(
            num_classes=NUM_CLASSES,
            names=list(classes.keys()),
            id=None
        ),
        length=-1,
        id=None
    )
}

features = Features(features_dict)
pre_tokenized_dataset = Dataset.from_list(examples, features=features)


** Batched Tokenization **
Note : The tokenizer does not touch the labels , it only does integer encoding to the words and splits them to sub-**words**


In [ ]:
TARGET_HUGGING_FACE_MODEL = "bert-base-uncased"
distilBertTokenizer = AutoTokenizer.from_pretrained(TARGET_HUGGING_FACE_MODEL)  # gives T : DistilBertTokenizerFast


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def batched_tokenization_and_label_alignment(t_batch, label_all_tokens=True):
    tokenized_inputs = distilBertTokenizer(t_batch["tokens"], truncation=True, is_split_into_words=True)

    adjusted_batch_labels = []
    for i, original_ner_labels in enumerate(t_batch["ner_tags"]):

        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list (mapping) the tokens/subwords to their actual word in the initial sentence.

        previous_word_idx = None
        adjusted_labels= []

        # print(original_labels)---> [28, 28, 28, 0, 1, 1, 1, 28, 28, 28, 18, 19, 28, 28, 28, 28, 28, 8, 9, 28, 28, 12, 13, 13, 28]
        # print(word_ids) ---------> [None, 0, 1, 2, 3, 4, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, None]

        for word_idx in word_ids:
            if word_idx is None:
                adjusted_labels.append(-100)

            elif word_idx != previous_word_idx:
                adjusted_labels.append(original_ner_labels[word_idx])

            else:
                adjusted_labels.append(original_ner_labels[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx
        adjusted_batch_labels.append(adjusted_labels)

    tokenized_inputs["labels"] = adjusted_batch_labels
    return tokenized_inputs


tokenized_dataset_II = pre_tokenized_dataset.map(
    batched_tokenization_and_label_alignment,
    batched = True,
)

print(tokenized_dataset_II[0])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'tokens': ['In', 're', 'Estate', 'of', 'Iddi', 'Makokha', 'Mwima', 'alias', 'Makokha', 'Mwima', '(', 'Deceased', ')', '[', '2021', ']', 'eKLR', 'REPUBLIC', 'OF', 'KENYA', 'IN', 'THE', 'HIGH', 'COURT', 'OF', 'KENYA', 'AT', 'KAKAMEGA', 'SUCCESSION', 'CAUSE', 'NO', '.', '661', 'OF', '2013', 'IN', 'THE', 'MATTER', 'OF', 'THE', 'ESTATE', 'OF', 'IDDI', 'MAKOKHA', 'MWIMA', 'alias', 'MAKOKHA', 'MWIMA', '(', 'DECEASED', ')', 'RULING', '1', '.'], 'ner_tags': [0, 0, 0, 0, 17, 18, 18, 18, 18, 18, 0, 0, 0, 0, 0, 0, 21, 22, 22, 22, 0, 0, 3, 4, 4, 4, 4, 4, 0, 13, 14, 14, 14, 14, 14, 0, 0, 0, 0, 0, 0, 0, 17, 18, 18, 18, 18, 18, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 1999, 2128, 3776, 1997, 8909, 4305, 5003, 3683, 15256, 12464, 9581, 14593, 5003, 3683, 15256, 12464, 9581, 1006, 10181, 1007, 1031, 25682, 1033, 23969, 20974, 3072, 1997, 7938, 1999, 1996, 2152, 2457, 1997, 7938, 2012, 10556, 27052, 29107, 8338, 3426, 2053, 1012, 5764, 2487, 1997, 2286, 1999, 1996, 3043, 1997, 1996, 3776, 1997, 8909, 4305,

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from transformers import DataCollatorForTokenClassification

class CustomDataCollator(DataCollatorForTokenClassification):
    def __call__(self, batch):
        # Extract tokens and labels from the batch
        tokens = [example['input_ids'] for example in batch]
        labels = [example['labels'] for example in batch]

        # Pad tokens and labels
        input_ids = pad_sequence(
            [torch.tensor(ids, dtype=torch.long) for ids in tokens],
            batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = pad_sequence(
            [torch.tensor(l, dtype=torch.long) for l in labels],
            batch_first=True, padding_value=-100  # -100 is the default ignore_index for CrossEntropyLoss
        )

        attention_mask = torch.tensor(
            [[1]*len(example) + [0]*(input_ids.size(1)-len(example)) for example in input_ids],
            dtype=torch.long
        )

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }


**Train test Split**

In [ ]:
TEST_SIZE = 0.2
TRAIN_SIZE  = (1 - TEST_SIZE)
VALIDATION_SIZE = 0.1  #This is 10% o the training data not the entire dataset.


#From a Dataset We create a DatasetDictionary which is a way of splitting datasets for hugging  face models.

tv_test_dataset_dict = tokenized_dataset_II.train_test_split(
    test_size= TEST_SIZE
)

train_val_dataset = tv_test_dataset_dict.get('train')

train_val_dataset_dict = train_val_dataset.train_test_split(
    test_size= VALIDATION_SIZE
)

test_dataset = tv_test_dataset_dict.get('test')




train_dataset = train_val_dataset_dict.get('train')
validation_dataset = train_val_dataset_dict.get('test')

print(f"Training size: {len(train_dataset)},Test size: {len(test_dataset)}, Validation size: {len(validation_dataset)}")


Training size: 1440,Test size: 400, Validation size: 160


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
batch_size = 16
from torch.utils.data import DataLoader
data_collator = DataCollatorForTokenClassification(distilBertTokenizer)


# Create DataLoaders for training, validation, and test sets
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    generator=torch.Generator(device=device),  # set the device as a string here
    collate_fn=data_collator,
)

In [ ]:
seqEval= datasets.load_metric("seqeval", trust_remote_code=True)

<ipython-input-48-ed84e5baec00>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  seqEval= datasets.load_metric("seqeval", trust_remote_code=True)


**Metrics Computation**

In [ ]:
ner_string_labels = list(classes.keys())
ner_string_labels[0:5]

['OUT', 'B-RESPONDENT', 'I-RESPONDENT', 'B-COURT', 'I-COURT']

In [ ]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    #operation along the third dimension
    pred_logits = np.argmax(pred_logits, axis=2)

    # the logits and the probabilities are in the same order,so we don’t need to apply the softmax

    predictions = []
    true_labels = []

    for pred_row, label_row in zip(pred_logits, labels):
        #start by clearing the following lists.
        pred_row_filtered = []
        label_row_filtered = []

        for pred, label in zip(pred_row, label_row):
            if label != -100:
                pred_row_filtered.append(ner_string_labels[pred])
                label_row_filtered.append(ner_string_labels[label])

        predictions.append(pred_row_filtered)
        true_labels.append(label_row_filtered)

    results = seqEval.compute(predictions=predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


**Training** **Arguments**

In [ ]:
#Model Development.

MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 2
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_GRAD_NORM = 10
WEIGHT_DECAY = 0.01

** Model Development**

In [ ]:
to_train_model = AutoModelForTokenClassification.from_pretrained(TARGET_HUGGING_FACE_MODEL, num_labels= NUM_CLASSES)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
from transformers import TrainingArguments, IntervalStrategy, DataCollatorForTokenClassification,Trainer

trainingArguments = TrainingArguments(
    "test-ner",
    evaluation_strategy=IntervalStrategy.EPOCH,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=TRAIN_BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay = WEIGHT_DECAY,
    optim='adamw_torch',  # Use AdamW optimizer
)


In [ ]:
#To note  our Trainer class uses GPUs by default (if they are available from PyTorch), so you don’t need to manually send the model to GPU

legalNERTrainer = Trainer(
    model=to_train_model,
    args=trainingArguments,
    train_dataset=train_dataloader.dataset,  # use the DataLoader's dataset attribute
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=distilBertTokenizer,
    compute_metrics=compute_metrics,
)


print(f"Device for legalNERTrainer: {legalNERTrainer.args.device}")
legalNERTrainer.get_train_dataloader().batch_sampler.sampler.generator.device



The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.


Device for legalNERTrainer: cpu


device(type='cpu')

In [ ]:
legalNERTrainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1440
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 270


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trained_model = to_train_model


model_save_path = '/content/drive/MyDrive/ColabNotebooks/court_smart_in_ner_model'
tokenizer_save_path = '/content/drive/MyDrive/ColabNotebooks/court_smart_in_ner_tokenizer'

# Saving the trained model and tokenizer to my Google Drive
trained_model.save_pretrained(model_save_path)
distilBertTokenizer.save_pretrained(tokenizer_save_path)

print("SUCCESSFULLY SAVED")


**Loading The trained  Model **

In [ ]:
id2label = {str(i):label for i, label in enumerate(classes)}
label2id = {label:str(i) for i, label in enumerate(classes)}
label2id

{'OUT': '0',
 'B-RESPONDENT': '1',
 'I-RESPONDENT': '2',
 'B-COURT': '3',
 'I-COURT': '4',
 'B-PETITIONER': '5',
 'I-PETITIONER': '6',
 'B-PROVISION': '7',
 'I-PROVISION': '8',
 'B-LAWYER': '9',
 'I-LAWYER': '10',
 'B-STATUTE': '11',
 'I-STATUTE': '12',
 'B-CASE_NUMBER': '13',
 'I-CASE_NUMBER': '14',
 'B-DATE': '15',
 'I-DATE': '16',
 'B-OTHER_PERSON': '17',
 'I-OTHER_PERSON': '18',
 'B-JUDGE': '19',
 'I-JUDGE': '20',
 'B-ORG': '21',
 'I-ORG': '22',
 'B-GPE': '23',
 'I-GPE': '24',
 'B-WITNESS': '25',
 'I-WITNESS': '26',
 'B-PRECEDENT': '27',
 'I-PRECEDENT': '28'}

In [ ]:
config_path = '/content/drive/MyDrive/ColabNotebooks/court_smart_in_ner_model/config.json'

# Loadng the existing configuration
with open(config_path, 'r') as f:
    config = json.load(f)

# For example, updating the number of labels for the configuration

config['num_labels'] = NUM_CLASSES
config['id2label'] = id2label
config['label2id'] = label2id


with open(config_path, 'w') as f:
    json.dump(config, f)

print("SUCCESSFULLY UPLDATED THE MODEL CONFIGURATION.")


In [ ]:
# Load the saved model and tokenizer from Google Drive

model = AutoModelForTokenClassification.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)

In [ ]:
from  transformers import pipeline

distilBert_ner_pipeline = pipeline("ner", model= model, tokenizer=tokenizer, aggregation_strategy="simple")

example = "Bill Gates is the Founder of Microsoft"

ner_results =  distilBert_ner_pipeline(example)

print(ner_results)

**Post Prediction : Reconstructing original Words**

In [ ]:
def post_process_ner_results(post_prediction_results):
    entities = []
    parent_entity_dict = None

    for item in post_prediction_results:
        label = item['entity']

        if label != 'O':  # Exclude 'O' which denotes non-entity
            if label.startswith('B-'):
                if parent_entity_dict is not None:
                    entities.append(parent_entity_dict)
                parent_entity_dict = {
                    'entity': label[2:], 'words': [item['word']]
                }
            elif label.startswith('I-'):
                if parent_entity_dict is not None:
                    parent_entity_dict['words'][-1] += ' ' + item['word']  # Concatenate with previous word

        elif parent_entity_dict is not None:#indicates an entity that has been started and has not been processed
            entities.append(parent_entity_dict)
            parent_entity_dict = None

    if parent_entity_dict is not None:
        entities.append(parent_entity_dict)

    return entities

# Example usage:
out = [
    {'entity': 'B-PER', 'score': 0.9974279, 'index': 1, 'word': 'bill', 'start': 0, 'end': 4},
    {'entity': 'I-PER', 'score': 0.99712795, 'index': 2, 'word': 'gates', 'start': 5, 'end': 10},
    {'entity': 'B-PER', 'score': 0.95886695, 'index': 7, 'word': 'james', 'start': 29, 'end': 38},
    {'entity': 'I-PER', 'score': 0.95886695, 'index': 8, 'word': 'peter', 'start': 29, 'end': 38}
]

extracted_entities = post_process_ner_results(out)

print(extracted_entities)

